# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree()
            

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.127201e+02     1.579428e+00
 * time: 0.6417460441589355
     1     1.123996e+01     8.663664e-01
 * time: 1.7622771263122559
     2    -1.147696e+01     1.010772e+00
 * time: 2.256387948989868
     3    -3.375341e+01     7.793925e-01
 * time: 2.9904589653015137
     4    -4.723700e+01     5.700594e-01
 * time: 3.731119155883789
     5    -5.699086e+01     2.238274e-01
 * time: 4.449737071990967
     6    -5.984438e+01     1.611374e-01
 * time: 4.93700909614563
     7    -6.093599e+01     5.506578e-02
 * time: 5.437256097793579
     8    -6.136127e+01     5.866265e-02
 * time: 5.921553134918213
     9    -6.161785e+01     4.257662e-02
 * time: 6.408218145370483
    10    -6.183220e+01     2.331378e-02
 * time: 6.9015960693359375
    11    -6.199496e+01     2.318528e-02
 * time: 7.3882341384887695
    12    -6.207337e+01     2.106353e-02
 * time: 7.873556137084961
    13    -6.213579e+01     1.372543e-02
 * time: 8.370383024215698
   

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671072
    AtomicLocal         -18.8557668
    AtomicNonlocal      14.8522629
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485375 
    Xc                  -19.3336822

    total               -62.261666459868